# Setup

In [1]:
!pip install tensorflow_hub
!pip install "tensorflow"
!pip install "packaging<20.10" 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import matplotlib.pyplot as plt
import os
import seaborn as sns

import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import img_to_array

from PIL import Image

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Load Data

In [4]:
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('cassava-images')

In [5]:
for item in bucket.objects.all():
    print(item)

s3.ObjectSummary(bucket_name='cassava-images', key='train.csv')
s3.ObjectSummary(bucket_name='cassava-images', key='train_images.zip')


In [6]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket = 'cassava-images', Key='train.csv')
train_df = pd.read_csv('s3://cassava-images/train.csv')

In [7]:
print(train_df)

             image_id  label
0      1000015157.jpg      0
1      1000201771.jpg      3
2       100042118.jpg      1
3      1000723321.jpg      1
4      1000812911.jpg      3
...               ...    ...
21392   999068805.jpg      3
21393   999329392.jpg      3
21394   999474432.jpg      1
21395   999616605.jpg      4
21396   999998473.jpg      4

[21397 rows x 2 columns]


In [8]:
from io import BytesIO
from zipfile import ZipFile

obj = bucket.Object('train_images.zip')
bytes_ = BytesIO(obj.get()["Body"].read())
z = ZipFile(bytes_)

In [9]:
def preprocess_fn(image):   
    image = Image.open(z.open(image))
    image_array = np.asarray(image)

    # Make the array contiguous
    image_array = np.ascontiguousarray(image_array)

    # Resize the image
    image_array = np.array(Image.fromarray(image_array).resize((224, 224)))

    image_array = image_array / 255.

    # Convert to tensor
    image_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)

    return image_tensor

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_df["image_id"], train_df["label"], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [11]:
# Subsample the training set
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.66, random_state=42)
# Subsample the validation set as well
X_val, _, y_val, _ = train_test_split(X_val, y_val, test_size=0.8, random_state=42)

In [12]:
print(X_train)

12675     327041212.jpg
9939     2767669424.jpg
7762     2381876074.jpg
7063     2261678500.jpg
2588     1451055508.jpg
              ...      
16771    4017977717.jpg
17907    4223515593.jpg
543      1095906723.jpg
8495     2506348944.jpg
4272     1762982108.jpg
Name: image_id, Length: 4364, dtype: object


In [13]:
# Training Set
list_image_train = [preprocess_fn(file) for file in X_train.values]
train_ds = tf.data.Dataset.from_tensor_slices((list_image_train)).batch(2)
train_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_train, dtype=tf.int64))).batch(2)

In [14]:
# Validation Set
list_image_val = [preprocess_fn(file) for file in X_val.values]
val_ds = tf.data.Dataset.from_tensor_slices((list_image_val)).batch(2)
val_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_val, dtype=tf.int64))).batch(2)

In [15]:
# Testing Set
list_image_test = [preprocess_fn(file) for file in X_test.values]
test_ds = tf.data.Dataset.from_tensor_slices((list_image_test)).batch(2)
test_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_test, dtype=tf.int64))).batch(2)

# Baseline Model

In [16]:
classifier = hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2')
predictions_list = []
for batch in test_ds:
    probabilities = classifier(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list.append(predictions)

In [17]:
from sklearn.metrics import accuracy_score

# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy)

Accuracy: 0.8691588785046729


In [18]:
zero_prediction_list = []
one_prediction_list = []
two_prediction_list = []
three_prediction_list = []
four_prediction_list = []

for i in range(len(all_labels)):
    if all_labels[i] == 0:
        zero_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 1:
        one_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 2:
        two_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 3:
        three_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 4:
        four_prediction_list.append(all_predictions[i])

zero_label_list = [0] * len(zero_prediction_list)
one_label_list = [1] * len(one_prediction_list)
two_label_list = [2] * len(two_prediction_list)
three_label_list = [3] * len(three_prediction_list)
four_label_list = [4] * len(four_prediction_list)

In [19]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_0 = f1_score(zero_label_list, zero_prediction_list, average='weighted')
f1_1 = f1_score(one_label_list, one_prediction_list, average='weighted')
f1_2 = f1_score(two_label_list, two_prediction_list, average='weighted')
f1_3 = f1_score(three_label_list, three_prediction_list, average='weighted')
f1_4 = f1_score(four_label_list, four_prediction_list, average='weighted')
print(f"Weighted F1 score for label 0: {f1_0}")
print(f"Weighted F1 score for label 1: {f1_1}")
print(f"Weighted F1 score for label 2: {f1_2}")
print(f"Weighted F1 score for label 3: {f1_3}")
print(f"Weighted F1 score for label 4: {f1_4}")

Weighted F1 score for label 0: 0.8493150684931507
Weighted F1 score for label 1: 0.8671875
Weighted F1 score for label 2: 0.835633626097867
Weighted F1 score for label 3: 0.9693582578531509
Weighted F1 score for label 4: 0.8717366628830874


# Fine Tuning - Frozen Layers

In [21]:
# Define the new model
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=False),
])

# Compile the model with an appropriate loss function and optimizer
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [22]:
# Combine the image and label datasets into a single dataset
train_ds_combined = tf.data.Dataset.zip((train_ds, train_labels))
val_ds_combined = tf.data.Dataset.zip((val_ds, val_labels))
history = model.fit(train_ds_combined, steps_per_epoch=len(train_ds_combined) / 10, epochs=10, validation_data=val_ds_combined)

Epoch 1/10
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


218/218 [==============================] - 12s 26ms/step - loss: 0.3346 - accuracy: 0.8836 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 2/10
218/218 [==============================] - 5s 21ms/step - loss: 0.4046 - accuracy: 0.8630 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 3/10
218/218 [==============================] - 5s 21ms/step - loss: 0.3900 - accuracy: 0.8836 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 4/10
218/218 [==============================] - 5s 21ms/step - loss: 0.3311 - accuracy: 0.8858 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 5/10
218/218 [==============================] - 5s 23ms/step - loss: 0.3507 - accuracy: 0.8790 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 6/10
218/218 [==============================] - 5s 22ms/step - loss: 0.4100 - accuracy: 0.8721 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 7/10
218/218 [==============================] - 5s 21ms/step - loss: 0.4449 - accuracy: 0.8470 - val_loss: 0.3815 - val_accuracy: 0.8621
Epoch 8/1

218/218 [==============================] - 5s 22ms/step - loss: 0.4014 - accuracy: 0.8436 - val_loss: 0.3815 - val_accuracy: 0.8621


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 6)                 4234118   
                                                                 
Total params: 4,234,118
Trainable params: 0
Non-trainable params: 4,234,118
_________________________________________________________________


In [24]:
predictions_list_fine_tuned = []
for batch in test_ds:
    probabilities = model.predict(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list_fine_tuned.append(predictions)

1/1 [==============================] - 0s 28ms/step


In [25]:
from sklearn.metrics import accuracy_score

# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list_fine_tuned, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy)

Accuracy: 0.8691588785046729


# Fine Tuning - Trainable Layers

In [26]:
# Define the new model
model_tunable = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=True),
    tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# Compile the model with an appropriate loss function and optimizer
model_tunable.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [27]:
history_tunable = model_tunable.fit(train_ds_combined, steps_per_epoch=len(train_ds_combined) // 10, epochs=10, validation_data=val_ds_combined)

Epoch 1/10
218/218 [==============================] - 29s 51ms/step - loss: 1.5804 - accuracy: 0.4427 - val_loss: 1.2710 - val_accuracy: 0.6297
Epoch 2/10
218/218 [==============================] - 10s 45ms/step - loss: 1.2780 - accuracy: 0.6124 - val_loss: 1.1362 - val_accuracy: 0.6297
Epoch 3/10
218/218 [==============================] - 10s 45ms/step - loss: 1.2522 - accuracy: 0.6032 - val_loss: 1.1074 - val_accuracy: 0.6297
Epoch 4/10
218/218 [==============================] - 10s 45ms/step - loss: 1.2101 - accuracy: 0.6170 - val_loss: 1.0951 - val_accuracy: 0.6297
Epoch 5/10
218/218 [==============================] - 10s 45ms/step - loss: 1.2708 - accuracy: 0.5688 - val_loss: 1.0936 - val_accuracy: 0.6297
Epoch 6/10
218/218 [==============================] - 10s 45ms/step - loss: 1.2511 - accuracy: 0.5849 - val_loss: 1.0652 - val_accuracy: 0.6297
Epoch 7/10
218/218 [==============================] - 10s 45ms/step - loss: 1.1367 - accuracy: 0.6422 - val_loss: 1.0878 - val_accuracy:

In [28]:
model_tunable.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 6)                 4234118   
                                                                 
 dense (Dense)               (None, 6)                 42        
                                                                 
Total params: 4,234,160
Trainable params: 4,209,760
Non-trainable params: 24,400
_________________________________________________________________


In [29]:
predictions_list_fine_tuned = []
for batch in test_ds:
    probabilities = model_tunable.predict(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list_fine_tuned.append(predictions)

print(predictions_list_fine_tuned)

1/1 [==============================] - 0s 27ms/step
[<tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf.Tensor: shape=(2,), dtype=int64, numpy=array([3, 3])>, <tf

In [30]:
from sklearn.metrics import accuracy_score

# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list_fine_tuned, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy_tunable = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy_tunable)

Accuracy: 0.624766355140187


In [31]:
print(all_predictions)
print(all_labels)

mispredictions = []
for i in range(len(all_labels)):
    if all_labels[i] != all_predictions[i]:
        mispredictions.append([all_labels[i], all_predictions[i]])

for j in all_predictions:
    print(all_predictions[j])

tf.Tensor([3 3 3 ... 3 3 3], shape=(4280,), dtype=int64)
tf.Tensor([4 3 3 ... 3 3 3], shape=(4280,), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(),

Observation: all predictions are 3. This is most likely due to the class imbalance - since a great majority of the dataset is of label 3, the model learns to guess 3 for all images. Therefore, we will oversample the minority classes.

# Addressing Class Imbalance

Ideally, we want to oversample the minority class. However, this runs into an issue due to the size of our dataset. If we are to oversample the minority classes then the size of the dataset will be incredibly large - too large for our memory. Therefore, we will take a random sample of the majority class. In practice, this could be an issue since we may miss out on important features.

In [21]:
train_df['label'].value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64

In [22]:
# Count the number of samples in each class
class_counts = train_df['label'].value_counts()

# Define the number of samples to keep from the majority class
num_samples_to_keep = 2200

# Select a subset of the majority class (label 3) to keep
majority_subset = train_df[train_df['label'] == 3].sample(num_samples_to_keep, random_state=42)

# Combine the minority class samples with the majority subset
minority_df = train_df[train_df['label'] != 3]
oversampled_df = pd.concat([minority_df, majority_subset], ignore_index=True)

# Shuffle the dataset
oversampled_df = oversampled_df.sample(frac=1, random_state=42)

In [23]:
oversampled_df['label'].value_counts()

4    2577
2    2386
3    2200
1    2189
0    1087
Name: label, dtype: int64

# Restarting Fine Tuning

In [24]:
X_train, X_test, y_train, y_test = train_test_split(oversampled_df["image_id"], oversampled_df["label"], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [25]:
# Subsample the training set
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.4, random_state=42)
# Subsample the validation set as well
X_val, _, y_val, _ = train_test_split(X_val, y_val, test_size=0.7, random_state=42)

In [26]:
# Training Set
list_image_train = []
list_image_train = [preprocess_fn(file) for file in X_train.values]
train_ds = tf.data.Dataset.from_tensor_slices((list_image_train)).batch(2)
train_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_train, dtype=tf.int64))).batch(2)

In [27]:
# Validation Set
list_image_val = []
list_image_val = [preprocess_fn(file) for file in X_val.values]
val_ds = tf.data.Dataset.from_tensor_slices((list_image_val)).batch(2)
val_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_val, dtype=tf.int64))).batch(2)

In [28]:
# Testing Set
list_image_test = []
list_image_test = [preprocess_fn(file) for file in X_test.values]
test_ds = tf.data.Dataset.from_tensor_slices((list_image_test)).batch(2)
test_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_test, dtype=tf.int64))).batch(2)

# Fine Tuning Frozen Layers

In [29]:
# Define the new model - 80.17% accuracy with 0.005 learning rate, 0.0001 l2 regularization
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=False),
    tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

# Compile the model with an appropriate loss function and optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [30]:
# Combine the image and label datasets into a single dataset
train_ds_combined = tf.data.Dataset.zip((train_ds, train_labels))
val_ds_combined = tf.data.Dataset.zip((val_ds, val_labels))

In [31]:
history = model.fit(train_ds_combined, epochs=5, validation_data=val_ds_combined)

Epoch 1/5
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


1879/1879 [==============================] - 24s 10ms/step - loss: 0.8335 - accuracy: 0.7495 - val_loss: 0.6338 - val_accuracy: 0.7827
Epoch 2/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.5918 - accuracy: 0.8060 - val_loss: 0.6211 - val_accuracy: 0.7843
Epoch 3/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.5855 - accuracy: 0.8070 - val_loss: 0.6198 - val_accuracy: 0.7812
Epoch 4/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.5839 - accuracy: 0.8070 - val_loss: 0.6194 - val_accuracy: 0.7812
Epoch 5/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.5833 - accuracy: 0.8073 - val_loss: 0.6193 - val_accuracy: 0.7812


In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 6)                 4234118   
                                                                 
 dense (Dense)               (None, 6)                 42        
                                                                 
Total params: 4,234,160
Trainable params: 42
Non-trainable params: 4,234,118
_________________________________________________________________


In [33]:
predictions_list_fine_tuned = []
for batch in test_ds:
    probabilities = model.predict(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list_fine_tuned.append(predictions)

1/1 [==============================] - 0s 27ms/step


In [34]:
from sklearn.metrics import accuracy_score

# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list_fine_tuned, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy)

Accuracy: 0.8022030651340997


In [35]:
zero_prediction_list = []
one_prediction_list = []
two_prediction_list = []
three_prediction_list = []
four_prediction_list = []

for i in range(len(all_labels)):
    if all_labels[i] == 0:
        zero_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 1:
        one_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 2:
        two_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 3:
        three_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 4:
        four_prediction_list.append(all_predictions[i])

zero_label_list = [0] * len(zero_prediction_list)
one_label_list = [1] * len(one_prediction_list)
two_label_list = [2] * len(two_prediction_list)
three_label_list = [3] * len(three_prediction_list)
four_label_list = [4] * len(four_prediction_list)

In [36]:
# calculate F1 score
f1_0 = f1_score(zero_label_list, zero_prediction_list, average='weighted')
f1_1 = f1_score(one_label_list, one_prediction_list, average='weighted')
f1_2 = f1_score(two_label_list, two_prediction_list, average='weighted')
f1_3 = f1_score(three_label_list, three_prediction_list, average='weighted')
f1_4 = f1_score(four_label_list, four_prediction_list, average='weighted')
print(f"Weighted F1 score for label 0: {f1_0}")
print(f"Weighted F1 score for label 1: {f1_1}")
print(f"Weighted F1 score for label 2: {f1_2}")
print(f"Weighted F1 score for label 3: {f1_3}")
print(f"Weighted F1 score for label 4: {f1_4}")

Weighted F1 score for label 0: 0.8225806451612903
Weighted F1 score for label 1: 0.8579088471849866
Weighted F1 score for label 2: 0.8741258741258742
Weighted F1 score for label 3: 0.969325153374233
Weighted F1 score for label 4: 0.888888888888889


In [37]:
confusion_matrix = tf.math.confusion_matrix(labels=all_labels, predictions=all_predictions)
print(confusion_matrix)

tf.Tensor(
[[153  16   3   6  41]
 [ 15 320  20  21  50]
 [  2  14 375  34  58]
 [  1   4  15 395   5]
 [ 35  32  17  24 432]], shape=(5, 5), dtype=int32)


# Data Augmentations

In [55]:
def data_augmentation(image):
    image = tf.image.random_flip_left_right(image, seed=123)
    image = tf.image.random_flip_up_down(image, seed=123)
    image = tf.image.random_brightness(image, max_delta=0.2, seed=123)    
    return image

def preprocess_fn_augmentation(image):   
    image = Image.open(z.open(image))
    
    image = data_augmentation(image)
    
    image_array = np.asarray(image)
    
    # Make the array contiguous
    image_array = np.ascontiguousarray(image_array)

    # Resize the image
    image_array = np.array(Image.fromarray(image_array).resize((224, 224)))

    image_array = image_array / 255.

    # Convert to tensor
    image_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)

    return image_tensor

In [56]:
# Training Set
list_image_train = [preprocess_fn_augmentation(file) for file in X_train.values]
train_ds = tf.data.Dataset.from_tensor_slices((list_image_train)).batch(2)
train_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_train, dtype=tf.int64))).batch(2)

In [14]:
# Validation Set
list_image_val = [preprocess_fn_augmentation(file) for file in X_val.values]
val_ds = tf.data.Dataset.from_tensor_slices((list_image_val)).batch(2)
val_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_val, dtype=tf.int64))).batch(2)

In [15]:
# Testing Set
list_image_test = [preprocess_fn_augmentation(file) for file in X_test.values]
test_ds = tf.data.Dataset.from_tensor_slices((list_image_test)).batch(2)
test_labels = tf.data.Dataset.from_tensor_slices((tf.cast(y_test, dtype=tf.int64))).batch(2)

## Baseline Model

In [69]:
classifier = hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2')
predictions_list = []
for batch in test_ds:
    probabilities = classifier(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list.append(predictions)

In [70]:
# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy)

Accuracy: 0.7911877394636015


In [71]:
zero_prediction_list = []
one_prediction_list = []
two_prediction_list = []
three_prediction_list = []
four_prediction_list = []

for i in range(len(all_labels)):
    if all_labels[i] == 0:
        zero_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 1:
        one_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 2:
        two_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 3:
        three_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 4:
        four_prediction_list.append(all_predictions[i])

zero_label_list = [0] * len(zero_prediction_list)
one_label_list = [1] * len(one_prediction_list)
two_label_list = [2] * len(two_prediction_list)
three_label_list = [3] * len(three_prediction_list)
four_label_list = [4] * len(four_prediction_list)

In [72]:
# calculate F1 score
f1_0 = f1_score(zero_label_list, zero_prediction_list, average='weighted')
f1_1 = f1_score(one_label_list, one_prediction_list, average='weighted')
f1_2 = f1_score(two_label_list, two_prediction_list, average='weighted')
f1_3 = f1_score(three_label_list, three_prediction_list, average='weighted')
f1_4 = f1_score(four_label_list, four_prediction_list, average='weighted')
print(f"Weighted F1 score for label 0: {f1_0}")
print(f"Weighted F1 score for label 1: {f1_1}")
print(f"Weighted F1 score for label 2: {f1_2}")
print(f"Weighted F1 score for label 3: {f1_3}")
print(f"Weighted F1 score for label 4: {f1_4}")

Weighted F1 score for label 0: 0.8288770053475936
Weighted F1 score for label 1: 0.8392370572207084
Weighted F1 score for label 2: 0.8661971830985915
Weighted F1 score for label 3: 0.969325153374233
Weighted F1 score for label 4: 0.8808290155440415


## Fine Tuned Model

In [61]:
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=False),
    tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

# Compile the model with an appropriate loss function and optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [62]:
# Combine the image and label datasets into a single dataset
train_ds_combined = tf.data.Dataset.zip((train_ds, train_labels))
val_ds_combined = tf.data.Dataset.zip((val_ds, val_labels))

In [63]:
history = model.fit(train_ds_combined, epochs=5, validation_data=val_ds_combined)

Epoch 1/5
1879/1879 [==============================] - 24s 9ms/step - loss: 0.9040 - accuracy: 0.7208 - val_loss: 0.6398 - val_accuracy: 0.7875
Epoch 2/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.6999 - accuracy: 0.7636 - val_loss: 0.6224 - val_accuracy: 0.7843
Epoch 3/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.6933 - accuracy: 0.7642 - val_loss: 0.6210 - val_accuracy: 0.7827
Epoch 4/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.6917 - accuracy: 0.7650 - val_loss: 0.6206 - val_accuracy: 0.7827
Epoch 5/5
1879/1879 [==============================] - 17s 9ms/step - loss: 0.6911 - accuracy: 0.7647 - val_loss: 0.6205 - val_accuracy: 0.7827


In [64]:
predictions_list_fine_tuned = []
for batch in test_ds:
    probabilities = model.predict(batch)
    predictions = tf.argmax(probabilities, axis=-1)
    predictions_list_fine_tuned.append(predictions)

1/1 [==============================] - 0s 27ms/step


In [65]:
from sklearn.metrics import accuracy_score

# Concatenate all the predictions into a single tensor
all_predictions = tf.concat(predictions_list_fine_tuned, axis=0)

# Concatenate all the labels into a single tensor
all_labels = tf.concat([labels for _, labels in enumerate(test_labels)], axis=0)

# Calculate the accuracy using TensorFlow's accuracy_score function
accuracy = accuracy_score(all_labels, all_predictions)

print('Accuracy:', accuracy)

Accuracy: 0.8017241379310345


In [66]:
zero_prediction_list = []
one_prediction_list = []
two_prediction_list = []
three_prediction_list = []
four_prediction_list = []

for i in range(len(all_labels)):
    if all_labels[i] == 0:
        zero_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 1:
        one_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 2:
        two_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 3:
        three_prediction_list.append(all_predictions[i])
    elif all_labels[i] == 4:
        four_prediction_list.append(all_predictions[i])

zero_label_list = [0] * len(zero_prediction_list)
one_label_list = [1] * len(one_prediction_list)
two_label_list = [2] * len(two_prediction_list)
three_label_list = [3] * len(three_prediction_list)
four_label_list = [4] * len(four_prediction_list)

In [67]:
# calculate F1 score
f1_0 = f1_score(zero_label_list, zero_prediction_list, average='weighted')
f1_1 = f1_score(one_label_list, one_prediction_list, average='weighted')
f1_2 = f1_score(two_label_list, two_prediction_list, average='weighted')
f1_3 = f1_score(three_label_list, three_prediction_list, average='weighted')
f1_4 = f1_score(four_label_list, four_prediction_list, average='weighted')
print(f"Weighted F1 score for label 0: {f1_0}")
print(f"Weighted F1 score for label 1: {f1_1}")
print(f"Weighted F1 score for label 2: {f1_2}")
print(f"Weighted F1 score for label 3: {f1_3}")
print(f"Weighted F1 score for label 4: {f1_4}")

Weighted F1 score for label 0: 0.8351063829787234
Weighted F1 score for label 1: 0.8670212765957447
Weighted F1 score for label 2: 0.8648648648648648
Weighted F1 score for label 3: 0.9642416769420468
Weighted F1 score for label 4: 0.888888888888889


In [68]:
confusion_matrix = tf.math.confusion_matrix(labels=all_labels, predictions=all_predictions)
print(confusion_matrix)

tf.Tensor(
[[157  16   3   5  38]
 [ 16 326  18  20  46]
 [  2  16 368  34  63]
 [  1   7  14 391   7]
 [ 39  33  15  21 432]], shape=(5, 5), dtype=int32)
